ModuleNotFoundError: No module named 'miniai'

ModuleNotFoundError: No module named 'miniai'

In [ ]:
# loading a dataloader - using a batch sampler and 4 workers
dls = DataLoaders.from_dd(tds, bs, num_workers=4)
dt = dls.train
xb,yb = next(iter(dt))
xb.shape,yb[:10]

In [4]:
# This is a very standard basic learner, which is not very flexible
# If we want to be able to change models quickly, test different, learning rates differently
# 

class Learner:
    # model, dataloader, loss function, learning rate, optimiser
    def __init__(self, model, dls, loss_func, lr, opt_func=optim.SGD):
        # storing all the attributes
        fc.store_attr()

    def one_batch(self):
        self.xb,self.yb = to_device(self.batch)
        self.preds = self.model(self.xb)
        self.loss = self.loss_func(self.preds, self.yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        with torch.no_grad(): self.calc_stats()

    def calc_stats(self):
        acc = (self.preds.argmax(dim=1)==self.yb).float().sum()
        self.accs.append(acc)
        n = len(self.xb)
        self.losses.append(self.loss*n)
        self.ns.append(n)
    
    # loop through each batch in a data learner and call the model
    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        for self.num,self.batch in enumerate(dl): self.one_batch()
        n = sum(self.ns)
        print(self.epoch, self.model.training, sum(self.losses).item()/n, sum(self.accs).item()/n)
    
    def fit(self, n_epochs):
        self.accs,self.losses,self.ns = [],[],[]
        self.model.to(def_device)
        # hardcoding a single learning rate
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.n_epochs = n_epochs
        for self.epoch in range(n_epochs):
            self.one_epoch(True)
            with torch.no_grad(): self.one_epoch(False)

In [1]:
# if you use @property decorator this means that you don't have to call the function
# you just call the property and it will output the value

# everytime we have a callback the callback points to this with_cbs() function
# which has a __call__() method taht returns the _f function which has 2 callbacks before and after the function being ran
# this allows us to do things before and after each learner method
class with_cbs:
    def __init__(self, nm): self.nm = nm
    def __call__(self, f):
        def _f(o, *args, **kwargs):
            try:
                o.callback(f'before_{self.nm}')
                f(o, *args, **kwargs)
                o.callback(f'after_{self.nm}')
            except globals()[f'Cancel{self.nm.title()}Exception']: pass
            finally: o.callback(f'cleanup_{self.nm}')
        return _f
     

In [2]:
# Basic callbacks learner

# self.callback('before_fit')
# passes in a list of our callbacks and the method name - method name being 'before_fit'
#

In [3]:
def run_cbs(cbs, method_nm, learn=None):
    for cb in sorted(cbs, key=attrgetter('order')):
        method = getattr(cb, method_nm, None)
        if method is not None: method(learn)
            
# will list our callbacks and order them

In [4]:
# Very simple callback with the method name written as a stringa and then will run them
# get attr
class CompletionCB(Callback):
    def before_fit(self, learn): self.count = 0
    def after_batch(self, learn): self.count += 1
    def after_fit(self, learn): print(f'Completed {self.count} batches')

NameError: name 'Callback' is not defined

In [5]:
class CompletionCB(Callback):
    def before_fit(self, learn): self.count = 0
    def after_batch(self, learn): self.count += 1
    def after_fit(self, learn): print(f'Completed {self.count} batches')

NameError: name 'Callback' is not defined

In [6]:
# This method is run at the end of the class
# it is responsible for looping through all of the callbacks in the sorted order that they have within their respective classe
# and then after we sorte them every single one will try to get the attribute taht we listed called method name
# this method name will run using method() as a callback as something that should be done
# now we can specify whatever kind of callbacks we want, before_fit, after_fit, before_batch, after_batch, after_fit, after_epcoh
# or any period after we do a specific thing any of these can be called
def run_cbs(cbs, method_nm, learn=None):
    for cb in sorted(cbs, key=attrgetter('order')):
        method = getattr(cb, method_nm, None)
        if method is not None: 
            method(learn)
            
# each method has a self.callback call running before and after it has done it's main loop
# callbacks can call exceptions incase we decide to not run batches / epochs depending on specific rules that we applied

In [7]:
class Metric:
    def __init__(self): 
        self.reset()
        
    def reset(self): 
        self.vals,self.ns = [],[]
        
    def add(self, inp, targ=None, n=1):
        self.last = self.calc(inp, targ)
        self.vals.append(self.last)
        self.ns.append(n)
        
    @property
    def value(self):
        ns = tensor(self.ns)
        return (tensor(self.vals)*ns).sum()/ns.sum()
    
    def calc(self, inps, targs): 
        return inps

In [8]:
class Accuracy(Metric):
    def calc(self, inps, targs): 
        return (inps==targs).float().mean()

In [9]:
# Torcheval means it has evaluation metrics for the pytroch library
from torcheval.metrics import MulticlassAccuracy, Mean

# Creating a context manager that will be able to handle the callbacks being called
# everything before the yield will be called before and anything after teh yield
# after the code is finished being donep
@contextmanager
def cb_ctx(self, nm):
    try:
        self.callback(f'before_{nm}')
        yield
        self.callback(f'after_{nm}')
    except globals()[f'Cancel{nm.title()}Exception']: pass
    finally: self.callback(f'cleanup_{nm}')

ModuleNotFoundError: No module named 'torcheval'

In [10]:
# __getattr__ then it will call this method if it doesn't exist
# instead get a callback passing in that name
# this means that we can handle these 4 as callbacks
# and now we can 
if self.training:
    self.backward()
    self.step()
    self.zero_grad()
    
def __getattr__(self, name):
    if name in ('predict','get_loss','backward','step','zero_grad'): 
        return partial(self.callback, name)
    raise AttributeError(name)
    
class TrainCB(Callback):
    def __init__(self, n_inp=1): self.n_inp = n_inp
    def predict(self, learn): learn.preds = learn.model(*learn.batch[:self.n_inp])
    def get_loss(self, learn): learn.loss = learn.loss_func(learn.preds, *learn.batch[self.n_inp:])
    def backward(self, learn): learn.loss.backward()
    def step(self, learn): learn.opt.step()
    def zero_grad(self, learn): learn.opt.zero_grad()

NameError: name 'self' is not defined

In [11]:
# We can then override our learner
# for example for zero_grad() instead of zeroing the gradients we can multiply them by a specific momentum
# this means that our previous gradients would still influence our given values, they would just reduce it a bit
# pytorch add the gradietns to teh existing gradients

# we can find a learning rate finder
# learning rate finders are useful to find the steepest gradient that we can have
# each bath we multiply the learning rate and continuously increase it to find out how it works


class LRFinderCB(Callback):
    def __init__(self, lr_mult=1.3): 
        fc.store_attr()
    
    def before_fit(self, learn):
        self.lrs, self.losses = [],[]
        self.min = math.inf

    def after_batch(self, learn):
        # do not need to do any validatoin in learning rate finding
        if not learn.training: 
            raise CancelEpochException()
        
        self.lrs.append(learn.opt.param_groups[0]['lr'])
        loss = to_cpu(learn.loss)
        # append the current loss
        self.losses.append(loss)
        
        # see how we want to modify the learning rate based on how the loss is right now
        if loss < self.min: 
            self.min = loss
        if loss > self.min*3: 
            raise CancelFitException()
        # change the learning rate by the multiplier
        for g in learn.opt.param_groups: 
            g['lr'] *= self.lr_mult

NameError: name 'Callback' is not defined

In [12]:
torch.optim.lr_scheduler

# a learning rate scheduler will decay the learning rate of the learner after every time you set step

# Actionvation stats allow us to look inside our models and diagnose how our model is doing
# torch.manual_seed(seed)
# randm.seed(seed)

NameError: name 'torch' is not defined

In [13]:
# High learning rates can cause the model to not train well
# we do not want activations close to 0 otherwise they will all just be dead units
# mean of 0 as long as they have a standard deviation around 1 or bigger we do not want a std of 0

# Hooks allow you to look inside our model and visualise how it is training
# they will be attached to a specific layer that you attach them to and then be called whenever that layer is cross either during the 
# forward pass or a backward pass depending on which kind of hook they are

# layer number, module, inputs and outputs

# go through each layer of the module and register_forward_hook
# after you register the forward hook which should be a partial(hook_function_name, layer index)
# it will call it whenever you run it

# hooks are a particular kind of callback

In [14]:
# Hook classes would simplify this
 
# make the hooks class a context manager 
with Hooks(model, append_stats) as hooks:
    fit(model)
    fig,axs = plt.subplots(1,2, figsize=(10,4))
    for h in hooks:
        for i in 0,1: axs[i].plot(h.stats[i])
    plt.legend(range(6));

    # classs for context manager
    # __enter__() - when it hits the with statement
    # __exit__() - when it finishes
class Hooks(list):
    def __init__(self, ms, f): super().__init__([Hook(m, f) for m in ms])
    def __enter__(self, *args): return self
    def __exit__ (self, *args): self.remove()
    def __del__(self): self.remove()
    def __delitem__(self, i):
        self[i].remove()
        super().__delitem__(i)
    def remove(self):
        for h in self: h.remove()

NameError: name 'Hooks' is not defined